In [ ]:
# Cell 1: Imports and Initial Setup
import pyspedas
from pyspedas import time_string
from pytplot import tplot, get_data
import numpy as np
import pandas as pd
import scipy.io.wavfile as wavfile
import os
from datetime import datetime
import sys, os
import astropy.units as u

# Keep your existing imports and directory setup code here
from data_management import *
from buttons import *

global save_dir

In [ ]:
# Cell 2: Set Save Directory
last_dir_file = "last_selected_dir.txt"
save_dir = set_save_directory(last_dir_file)

show_directory_button(save_dir)

print(f'{datetime.now().strftime("%Y-%m-%d %H:%M:%S")} - 🛟 Save Directory Set: {save_dir}')

In [ ]:
# Cell 3: Helper Functions
def interpolate_nans(data):
    """Interpolate NaN values in data."""
    nan_mask = np.isnan(data)
    data[nan_mask] = np.interp(np.flatnonzero(nan_mask), np.flatnonzero(~nan_mask), data[~nan_mask])
    return data

def normalize_to_int16(data):
    """Normalize data to int16 range."""
    data = interpolate_nans(data)
    data_max, data_min = np.max(data), np.min(data)
    if data_max == data_min:
        return np.zeros(data.shape, dtype=np.int16)
    normalized_data = (2 * (data - data_min) / (data_max - data_min) - 1) * 32767
    return normalized_data.astype(np.int16)

def audify_mag_data(trange, save_dir, fs_aud, bmag, br, bt, bn, rsquared_normalized=False, custom_suffix=''):
    """Audify magnetic field data."""
    print("Starting audification process...")

    component_mapping = {'|B|': '001', 'Br': '002', 'Bt': '003', 'Bn': '004'}
    components = {'|B|': bmag, 'Br': br, 'Bt': bt, 'Bn': bn}

    start_time = pd.to_datetime(trange[0])
    end_time = pd.to_datetime(trange[1])

    for component, data in components.items():
        audio_data = normalize_to_int16(data)
        suffix = "_rsquared_normalized" if rsquared_normalized else ""
        file_name = f"{save_dir}/PSP_FIELDS_{component_mapping[component]}_{start_time.strftime('%Y-%m-%d_%H%M')}_to_{end_time.strftime('%m-%d_%H%M')}_{component}{suffix}{custom_suffix}.wav"
        wavfile.write(file_name, fs_aud, audio_data)
        print(f"Saved audio file: {file_name}")

    print("Audification process completed.")

# Cell 3: Helper Functions (modified)

def generate_markers(times, trange, markers_per_hour, save_dir, custom_suffix=''):
    """Generate markers for the audified data."""
    print(f"Generating markers for time range: {trange[0]} to {trange[1]}")
    print(f"Number of time points: {len(times)}")
    
    start_time = pd.to_datetime(trange[0])
    end_time = pd.to_datetime(trange[1])
    
    times_datetime = pd.to_datetime(times, unit='s')
    
    print(f"Data time range: {times_datetime.min()} to {times_datetime.max()}")
    
    marker_times = pd.date_range(start=start_time, end=end_time, freq=f'{60/markers_per_hour}min')
    
    closest_indices = np.searchsorted(times_datetime, marker_times)
    
    valid_markers = closest_indices < len(times)
    marker_times = marker_times[valid_markers]
    closest_indices = closest_indices[valid_markers]
    
    print(f"Total markers generated: {len(marker_times)}")
    
    filename = f"{save_dir}/PSP_FIELDS_MARKER_SET_{start_time.strftime('%Y-%m-%d_%H%M')}_to_{end_time.strftime('%Y-%m-%d_%H%M')}_{markers_per_hour}_per_hour{custom_suffix}.txt"
    
    with open(filename, 'w') as f:
        for marker_time, sample_number in zip(marker_times, closest_indices):
            time_str = marker_time.strftime('%H:%M:%S.%f')[:12]
            date_str = marker_time.strftime('(%Y-%m-%d)')
            f.write(f"{time_str} {date_str}\t{sample_number}\n")
    
    print(f"Marker file created: {filename}")
    return filename

In [ ]:
# Cell 4: Data Download and Processing
# Define the requested date range
trange = ['2023-03-16/00:00:00', '2023-03-18/00:00:00']

# Load magnetic field data
mag_vars = pyspedas.psp.fields(trange=trange, datatype='mag_rtn', level='l2', time_clip=True, get_support_data=True)

# Load solar wind proton data (which includes sun distance)
spi_vars = pyspedas.psp.spi(trange=trange, datatype='spi_sf00_l3_mom', level='l3', time_clip=True)

# Get magnetic field and time data
mag_data = get_data('psp_fld_l2_mag_RTN')
times, mag_field = mag_data.times, mag_data.y

# Get sun distance and time data
sun_dist_data = get_data('psp_spi_SUN_DIST')
sun_dist_time, sun_dist = sun_dist_data.times, sun_dist_data.y

In [ ]:
# Cell 5: Solar Distance Calculation and R-squared Normalization

# Create an interpolation function
sun_dist_interp = interpolate.interp1d(sun_dist_time, sun_dist, kind='linear', bounds_error=False, fill_value="extrapolate")

# Interpolate sun distance to match magnetic field data times
sun_dist_interpolated = sun_dist_interp(times)

# Convert kilometers to astronomical units (AU)
au_in_km = 149597870.7
sun_dist_au = sun_dist_interpolated / au_in_km

# Calculate R squared (distance in AU squared)
r_squared = sun_dist_au ** 2

# Normalize magnetic field components
normalized_mag_field = mag_field * r_squared[:, np.newaxis]

print(f"Normalized magnetic field shape: {normalized_mag_field.shape}")

# Calculate the magnitude of the original and normalized magnetic field
mag_field_magnitude = np.linalg.norm(mag_field, axis=1)
normalized_mag_field_magnitude = np.linalg.norm(normalized_mag_field, axis=1)

print(f"Original B magnitude range: {mag_field_magnitude.min():.2f} - {mag_field_magnitude.max():.2f} nT")
print(f"Normalized B magnitude range: {normalized_mag_field_magnitude.min():.2f} - {normalized_mag_field_magnitude.max():.2f} nT*AU^2")

# Extract normalized magnetic field components
br_normalized, bt_normalized, bn_normalized = normalized_mag_field[:, 0], normalized_mag_field[:, 1], normalized_mag_field[:, 2]
bmag_normalized = normalized_mag_field_magnitude

In [ ]:
# Cell 6: Main Execution (modified)
trange = ['2023-03-16/11:00:00', '2023-03-17/07:00:00']

# Set audio sampling rate and markers per hour
fs_aud = 22000
markers_per_hour = 1

# Set R-squared normalization flag
rsquared_normalized = True  # Set to False for normal data, True for R-squared normalized data

# Set custom suffix
custom_suffix = '_SA_STREAM'  # You can modify this line to add your desired suffix

# Convert trange to datetime objects
start_time = pd.to_datetime(trange[0])
end_time = pd.to_datetime(trange[1])

# Filter data based on the new time range
mask = (times >= start_time.timestamp()) & (times <= end_time.timestamp())
times_filtered = times[mask]

# Filter and use normalized or original data based on the flag
if rsquared_normalized:
    br_to_use = br_normalized[mask]
    bt_to_use = bt_normalized[mask]
    bn_to_use = bn_normalized[mask]
    bmag_to_use = bmag_normalized[mask]
else:
    br_to_use = br[mask]
    bt_to_use = bt[mask]
    bn_to_use = bn[mask]
    bmag_to_use = mag_field_magnitude[mask]

# Generate markers
marker_file = generate_markers(times_filtered, trange, markers_per_hour, save_dir, custom_suffix)

# Audify the magnetic field data with custom suffix
audify_mag_data(trange, save_dir, fs_aud, bmag_to_use, br_to_use, bt_to_use, bn_to_use, rsquared_normalized, custom_suffix)

# Print summary
print(f"Data processed for time range: {trange[0]} to {trange[1]}")
print(f"Marker file created: {marker_file}")
print(f"R-squared normalization: {'Applied' if rsquared_normalized else 'Not applied'}")
print(f"Custom suffix applied: '{custom_suffix}'")